In [633]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

import sys; sys.path.append('/home/mitch/school/mfl/')
import src.json_utils

In [634]:
raw = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim2/2005/'

figs = '/home/mitch/github/mfl/fig/'
dicts= '/home/mitch/github/mfl/dicts/'

In [635]:
education = src.json_utils.load_json(dicts + 'education.json')
rename = src.json_utils.load_json(dicts + 'rename_ind_2005.json')

In [636]:
ids = ['folio', 'ls', 'pid_link']
keep = ['folio', 'ls', 'pid_link']
os.chdir(raw + 'book IIIB/data/')
portad = pd.read_stata('iiib_portad.dta')
portad = portad.sort_values(by=['folio', 'ls'])

In [637]:
portad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20607 entries, 0 to 20606
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   catorcena  20607 non-null  float32
 1   edad       20606 non-null  float32
 2   edo_civil  20607 non-null  int8   
 3   ent        20607 non-null  float32
 4   folio      20607 non-null  object 
 5   ls         20607 non-null  object 
 6   pid_link   20607 non-null  object 
 7   rel        20607 non-null  float32
dtypes: float32(4), int8(1), object(3)
memory usage: 986.1+ KB


In [638]:
os.chdir(raw + 'book IIIA/data/')
weights_3a = pd.read_stata('weights.dta')
weights_3a = (weights_3a.query('fac_3a>0')
              .drop_duplicates()
              .sort_values(by=['folio', 'ls']))
weights_3a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19544 entries, 0 to 35087
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   folio   19544 non-null  object
 1   ls      19544 non-null  object
 2   fac_3a  19544 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 534.4+ KB


In [639]:
# ids
os.chdir(raw + 'book C/data/')
df_ls = pd.read_stata('c_ls.dta')
df_ls = df_ls.drop(columns='pid_link')

os.chdir(raw + 'book IIIA/data/')
# employment
df_tb = pd.read_stata('iiia_tb.dta')
df_tb = df_tb.drop(columns='pid_link')

# education
df_ed = pd.read_stata('iiia_ed.dta')
df_ed = df_ed.drop(columns='pid_link')

# non labor income, is ANNUAL
df_iin = pd.read_stata('iiia_iin.dta')
df_iin = df_iin.drop(columns='pid_link')

os.chdir(raw + 'book IIIB/data/')
# credit
df_cr = pd.read_stata('iiib_cr.dta')
df_cr = df_cr.drop(columns='pid_link')

# credit 1
df_cr1 = pd.read_stata('iiib_cr1.dta')
df_cr1 = df_cr1.drop(columns='pid_link')

# family structure (gender)
df_thi = pd.read_stata('iiib_thi.dta')
df_thi = df_thi.drop(columns='pid_link')

In [640]:
df = (portad.merge(weights_3a, on=['folio', 'ls'], how='left')
        .merge(df_ls, on=['folio', 'ls'], how='left')
        .merge(df_tb, on=['folio', 'ls'], how='left')
        .merge(df_ed, on=['folio', 'ls'], how='left')
        .merge(df_cr, on=['folio', 'ls'], how='left')
        .merge(df_cr1, on=['folio', 'ls'], how='left')
        .merge(df_thi, on=['folio', 'ls'], how='left')
        )

In [641]:
df = df.rename(columns=rename)
df = df[rename.values()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21057 entries, 0 to 21056
Data columns (total 50 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hhid                        21057 non-null  object 
 1   hhmember                    21057 non-null  object 
 2   weight_ind                  19464 non-null  float64
 3   is_hhm                      20986 non-null  float32
 4   age                         21056 non-null  float32
 5   attended_school             21004 non-null  float32
 6   education                   18858 non-null  float32
 7   gender                      21043 non-null  float32
 8   income_type                 6991 non-null   float64
 9   income_undetailed           5522 non-null   float32
 10  has_income_second_job       127 non-null    float32
 11  income_second_job           110 non-null    float32
 12  has_wage                    21005 non-null  object 
 13  wage                        950

In [642]:
df = df[~df.weight_ind.isna()]

# Household Size

In [643]:
# define household size
df['hh_size'] = df.groupby('hhid')['hhmember'].transform('size')

# Education

In [644]:
df['attended_school'] = df['attended_school'].replace({1.0:1.0, 3.0:0.0}).astype(bool)
df.attended_school.value_counts(dropna=False)

True     17387
False     2077
Name: attended_school, dtype: int64

In [645]:
education

{'1.0': 'Low',
 '2.0': 'Low',
 '3.0': 'Low',
 '4.0': 'Medium',
 '5.0': 'Medium',
 '6.0': 'Medium',
 '7.0': 'Medium',
 '8.0': 'Medium',
 '9.0': 'High',
 '10.0': 'High',
 '98.0': 'Low'}

In [646]:
df.education.value_counts(dropna=False)

3.0     7289
4.0     4837
6.0     2687
NaN     2081
9.0     1722
5.0      263
8.0      241
7.0      170
10.0      78
1.0       76
2.0       15
98.0       5
Name: education, dtype: int64

In [647]:
df.education = df.education.astype('str').replace(education)
df.loc[df.attended_school == False, 'education'] = 'low'
df.education.value_counts()

Medium    8198
Low       7385
low       2077
High      1800
nan          4
Name: education, dtype: int64

# Age

In [648]:
df.age.isna().sum()

1

# Gender

In [649]:
df.gender.value_counts(dropna=False)

1.0    10719
3.0     5188
2.0     3545
NaN       12
Name: gender, dtype: int64

In [650]:
df['gender'] = df['gender'].replace(
    {1.0:'Female',
     2.0:'Male',
     3.0:'Male'}
    )
df.gender.value_counts(dropna=False)

Female    10719
Male       8733
NaN          12
Name: gender, dtype: int64

# Income

In [651]:
detailed_income_sources = ['wage', 'piecework', 'tips', 'extrahours', 'christmasbonus', 'bonus', 'profitdist', 'other']
has_values = {
    'wage':'A',
    'piecework':'B',
    'tips':'C',
    'extrahours':'D',
    'christmasbonus':'E',
    'bonus':'F',
    'profitdist':'H',
    'other':'M'
}

In [652]:
df.income_type.value_counts(dropna=False)

NaN    13139
3.0     5080
1.0      859
8.0      386
Name: income_type, dtype: int64

In [653]:
df.income_type = df.income_type.replace({
    3.0:'undetailed',
    1.0:'detailed',
    8.0:'dk'
})

In [654]:
df['income_main_job'] = 0.00

undetailed = df['income_type'] == 'undetailed'
detailed   = df['income_type'] == 'detailed'
dk_income  = df['income_type'] == 'dk'

df.loc[df.income_type == 'undetailed', 'income_main_job'] = df.loc[undetailed, 'income_undetailed']

In [655]:
for detailed_income_source in detailed_income_sources:
    knows = df['has_' + detailed_income_source] == has_values[detailed_income_source]
    dk = df['has_' + detailed_income_source] == 8

    df.loc[dk, detailed_income_source] = np.average(df.loc[knows, detailed_income_source],
                                                    weights = df.loc[knows, 'weight_ind'])

    df[detailed_income_source] = df[detailed_income_source].fillna(0.00)
df.loc[df.income_type == 'detailed', 'income_main_job'] = df.loc[detailed, detailed_income_sources].sum(axis=1)

In [656]:
knows = df['has_' + 'wage'] == has_values['wage']
np.average(df.loc[knows, 'wage'], weights=df.loc[knows, 'weight_ind'])

68329.63311552304

In [657]:
income_sources = [
    'income_main_job',
    'income_second_job',
    'income_main_business',
    'income_second_business'
]
df['income'] = df[income_sources].sum(axis=1)
df['log_income'] = np.log(df['income'])

/home/mitch/anaconda3/envs/main/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Debts

In [658]:
df['has_debts'] = df['has_debts'].replace(
    {1.0:1, 8.0:0}
    ).fillna(0)

no_debts = df['has_debts'] == 0
df.loc[no_debts, 'debts'] = 0.0

df['has_debts'] = (df['debts'] > 0).astype(bool)
df.has_debts.value_counts()

False    14408
True      5056
Name: has_debts, dtype: int64

# Loans

In [659]:
df.asked_for_loan_12mth.value_counts(dropna=False)

NaN    12476
3.0     5363
1.0     1625
Name: asked_for_loan_12mth, dtype: int64

In [660]:
df['asked_for_loan_12mth'] = df['asked_for_loan_12mth'].replace(
    {1.0:1, 3.0:0, np.nan:0}
)
df['asked_for_loan_12mth'] = df['asked_for_loan_12mth'].astype(bool)
df.asked_for_loan_12mth.value_counts(dropna=False)

False    17839
True      1625
Name: asked_for_loan_12mth, dtype: int64

In [661]:
loan_types = {
    1.0:'bank',
    2.0:'savings_fund',
    3.0:'moneylender',
    4.0:'relative',
    5.0:'friends',
    6.0:'work',
    7.0:'pawnshop',
    8.0:'verbal_agreement_credit_program',
    9.0:'other_govt',
    10.0:'other',
    -1.0: np.nan
}
df['loan_type'] = df['loan_type'].replace(loan_types)
df.loan_type.value_counts(dropna=False)

NaN                                17841
relative                             389
friends                              316
savings_fund                         231
moneylender                          166
work                                 142
bank                                 122
other                                121
pawnshop                              99
other_govt                            28
verbal_agreement_credit_program        9
Name: loan_type, dtype: int64

In [662]:
got_loan = df['received_loan'] == 1.0
df.loc[~got_loan, 'loan'] = 0.0

df['has_loan'] = ~df.loan_type.isna() & got_loan
df.has_loan.value_counts()

False    17851
True      1613
Name: has_loan, dtype: int64

In [663]:
formal = ['bank', 'savings_fund', 'moneylender', 'pawnshop']
informal = ['relative', 'friends', 'work', 'verbal_agreement_credit_program', 'other_govt', 'other']
friends_relatives = ['relative', 'friends']

df['has_loan_formal']            = df['loan_type'].isin(formal)
df['has_loan_informal']          = df['loan_type'].isin(informal)
df['has_loan_friends_relatives'] = df['loan_type'].isin(friends_relatives)

df['loan_formal']   = df['loan']*df['has_loan_formal'] 
df['loan_informal'] = df['loan']*df['has_loan_informal'] 
df['loan_friends_relatives']   = df['loan']*df['has_loan_friends_relatives'] 

# Savings

In [664]:
df.has_savings.value_counts(dropna=False)

3.0    17042
1.0     2156
NaN      139
7.0      127
Name: has_savings, dtype: int64

In [665]:
has_savings = df['has_savings'] == 1.0
no_savings = df['has_savings'] == 3.0
dk_savings = df['has_savings'] == 7.0

#df.loc[has_savings, 'savings'] = df.loc[df.has_savings, 'savings']
df.loc[no_savings, 'savings'] = 0.0
df.loc[dk_savings, 'savings'] = 0.0

In [666]:
df['has_savings'] = df['has_savings'].replace({
    1.0   : True,
    3.0   : False,
    7.0   : False,
    np.nan: False
})
df.has_savings.value_counts(dropna=False)

False    17308
True      2156
Name: has_savings, dtype: int64

In [667]:
savings_locations = {
   'has_savings_none'          : 1.0,
   'has_savings_bank'          : 2.0,
   'has_savings_coop'          : 3.0,
   'has_savings_savingsbank'   : 4.0,
   'has_savings_friend_not_hhm': 5.0,
   'has_savings_afores'        : 6.0,
   'has_savings_caja_solidaria': 7.0,
   'has_savings_house'         : 8.0,
   'has_savings_work'          : 9.0,
   'has_savings_other'         : 10.0
}

In [668]:
for savings_type in list(savings_locations.keys()):
    df[savings_type] = (df[savings_type] == savings_locations[savings_type]).astype(bool)

In [669]:
df.loc[dk_savings, list(savings_locations.keys())].sum()

has_savings_none              0
has_savings_bank              0
has_savings_coop              0
has_savings_savingsbank       0
has_savings_friend_not_hhm    0
has_savings_afores            0
has_savings_caja_solidaria    0
has_savings_house             0
has_savings_work              0
has_savings_other             0
dtype: int64

In [670]:
"""
df['has_savings_none']              = df['has_savings_none'] == 1.0
df['has_savings_bank']              = df['has_savings_bank'] == 2.0
df['has_savings_coop']              = df['has_savings_coop'] == 3.0
df['has_savings_savingsbank']       = df['has_savings_savingsbank'] == 4.0
df['has_savings_friend_not_hhm']    = df['has_savings_friend_not_hhm'] == 5.0
df['has_savings_afores']            = df['has_savings_afores'] == 6.0
df['has_savings_caja_solidaria']    = df['has_savings_caja_solidaria'] == 7.0
df['has_savings_house']             = df['has_savings_house'] == 8.0
df['has_savings_work']              = df['has_savings_work'] == 9.0
df['has_savings_other']             = df['has_savings_other'] == 10.0
"""

"\ndf['has_savings_none']              = df['has_savings_none'] == 1.0\ndf['has_savings_bank']              = df['has_savings_bank'] == 2.0\ndf['has_savings_coop']              = df['has_savings_coop'] == 3.0\ndf['has_savings_savingsbank']       = df['has_savings_savingsbank'] == 4.0\ndf['has_savings_friend_not_hhm']    = df['has_savings_friend_not_hhm'] == 5.0\ndf['has_savings_afores']            = df['has_savings_afores'] == 6.0\ndf['has_savings_caja_solidaria']    = df['has_savings_caja_solidaria'] == 7.0\ndf['has_savings_house']             = df['has_savings_house'] == 8.0\ndf['has_savings_work']              = df['has_savings_work'] == 9.0\ndf['has_savings_other']             = df['has_savings_other'] == 10.0\n"

In [671]:
# restrict to people with a well defined age
df = (df.query('age >= 20')
      .query('age <= 65')
)

In [672]:
get_sum = [
    'income',
    'savings',
    'debts'
]

get_any = [
    'has_savings',
    'has_debts', 
    'has_loan',
    'has_loan_formal',
    'has_loan_informal',
    'has_loan_friends_relatives',
    'has_savings_none',
    'has_savings_bank',
    'has_savings_coop',
    'has_savings_savingsbank',
    'has_savings_friend_not_hhm',
    'has_savings_afores',
    'has_savings_caja_solidaria',
    'has_savings_house',
    'has_savings_work',
    'has_savings_other'
]

# get sum over set of individuals in household
for var in get_sum:
    df[var] = df.groupby('hhid')[var].transform('sum')

# check if any individual in household satisfies it
for var in get_any:
    df[var] = df.groupby('hhid')[var].transform(lambda x : x.sum() > 0)

In [673]:
# create single observation for each household by restricting to household head
df = df[df['is_hhm'] == 1.0].reset_index(drop=True)

# Income Outliers

In [674]:
from statsmodels.stats.weightstats import DescrStatsW
q_995 = DescrStatsW(df['income'], df['weight_ind']).quantile(0.995).iloc[0]
df = df[df['income'] < q_995]

In [675]:
os.chdir(interim)
df.to_csv('ind_2005.csv', index=False)

In [676]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5089 entries, 0 to 5111
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hhid                        5089 non-null   object 
 1   hhmember                    5089 non-null   object 
 2   weight_ind                  5089 non-null   float64
 3   is_hhm                      5089 non-null   float32
 4   age                         5089 non-null   float32
 5   attended_school             5089 non-null   bool   
 6   education                   5089 non-null   object 
 7   gender                      5089 non-null   object 
 8   income_type                 2592 non-null   object 
 9   income_undetailed           2054 non-null   float32
 10  has_income_second_job       54 non-null     float32
 11  income_second_job           45 non-null     float32
 12  has_wage                    5087 non-null   object 
 13  wage                        5089 